In [1]:
import cifrum as lib
import pandas as pd
import numpy as np

## Asset Creation

The library can create an `asset` in several ways. The method `portfolio_asset` accepts parameters:

- either `name: str`, or `names: List[str]` of asset(s) to be created
- optional `start_period: str` and `end_period: str` to define date range. If nothing is provided then the library takes maximum possible range 
- optional `currency: str` to compute further activities. If no is provided then the library takes the default value for the financial symbol

In [2]:
asset_name = 'us/SPY'

In [3]:
def print_asset(asset):
    print('== {}'.format(asset.symbol.identifier_str))
    print(asset.currency)
    print(asset._period_min)

In [4]:
print_asset(lib.portfolio_asset(name=asset_name))

== us/SPY
Currency(USD)
1993-01


In [5]:
print_asset(lib.portfolio_asset(name=asset_name, start_period='2010-1'))

== us/SPY
Currency(USD)
2010-01


In [6]:
print_asset(lib.portfolio_asset(name=asset_name, start_period='2010-1', end_period='2013-1'))

== us/SPY
Currency(USD)
2010-01


In [7]:
asset=lib.portfolio_asset(name=asset_name, 
                        start_period='2010-1', end_period='2013-1', 
                        currency='usd')
print_asset(asset)

== us/SPY
Currency(USD)
2010-01


## Basic Activities

### Close

Every asset has the `close` values. The `close` method returns `TimeSeries` object that wraps `numpy` array with additional meta-information: `start_period`, `end_period`, and `TimeSeriesKind`

In [8]:
close = asset.close()
close

TimeSeries(start_period=2010-01, end_period=2013-01, kind=TimeSeriesKind.VALUES, values=[ 88.3575  91.1138  96.6608  98.158   90.3561  85.682   91.5341  87.4168
  95.2452  98.884   98.886  105.4947 107.9527 111.7027 111.7158 114.9517
 113.662  111.7449 109.5096 103.4891  96.3049 106.8163 106.3822 107.4936
 112.4785 117.3609 121.1356 120.3268 113.1005 117.6946 119.0827 122.066
 125.1604 122.8828 123.5824 124.682  131.0646]

In [9]:
close.values

array([ 88.3575,  91.1138,  96.6608,  98.158 ,  90.3561,  85.682 ,
        91.5341,  87.4168,  95.2452,  98.884 ,  98.886 , 105.4947,
       107.9527, 111.7027, 111.7158, 114.9517, 113.662 , 111.7449,
       109.5096, 103.4891,  96.3049, 106.8163, 106.3822, 107.4936,
       112.4785, 117.3609, 121.1356, 120.3268, 113.1005, 117.6946,
       119.0827, 122.066 , 125.1604, 122.8828, 123.5824, 124.682 ,
       131.0646])

In [10]:
close.start_period, close.end_period

(Period('2010-01', 'M'), Period('2013-01', 'M'))

### Return

Next thing we derive from the `Close` values is [the `Return`](https://okama.io/#/glossary?page=cumulative-return). The `get_return` produces `TimeSeries` object that has one-month shorter `start_period`

In [11]:
assert asset.get_return().start_period == asset.close().start_period + 1
asset.get_return()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.DIFF, values=[ 3.11948618e-02  6.08799106e-02  1.54892159e-02 -7.94830783e-02
 -5.17297670e-02  6.83002264e-02 -4.49810508e-02  8.95525803e-02
  3.82045499e-02  2.02257190e-05  6.68315029e-02  2.32997487e-02
  3.47374359e-02  1.17275590e-04  2.89654642e-02 -1.12194948e-02
 -1.68666749e-02 -2.00035975e-02 -5.49769153e-02 -6.94198713e-02
  1.09147094e-01 -4.06398649e-03  1.04472365e-02  4.63739237e-02
  4.34074067e-02  3.21631821e-02 -6.67681507e-03 -6.00556152e-02
  4.06196259e-02  1.17940840e-02  2.50523376e-02  2.53502204e-02
 -1.81974490e-02  5.69322965e-03  8.89770712e-03  5.11910300e-02]

`get_return` accepts parameters as follows:

- `kind: str` that determines the time series reduction. It should be one of `values` (by default), `cumulative` or `ytd` values
- `real: bool` that states if the return should be adjusted to corresponding inflation (`False` by default)

For example, the call as follows gets the real cumulative return for the period:

In [12]:
asset.get_return(kind='cumulative', real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.CUMULATIVE, values=[ 0.03093816  0.08922921  0.10418255  0.0156316  -0.03596588  0.02966061
 -0.01801074  0.0693066   0.10877855  0.10833436  0.18037813  0.20215478
  0.2378109   0.22600133  0.25344211  0.23357638  0.21407032  0.18873133
  0.12028879  0.04093835  0.15694051  0.15321203  0.16814171  0.21695841
  0.26421707  0.29504247  0.28252121  0.20691432  0.25778264  0.27469479
  0.29939772  0.32641926  0.3027886   0.31644298  0.33174269  0.39578862]

### CAGR

Asset supports computation of [CAGR](https://okama.io/#/glossary?page=compound-annual-growth-rate) activity:

In [13]:
asset.cagr()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.14046159]

The asset can also compute the real CAGR:

In [14]:
asset.cagr(real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.1175661]

### Risk

The Risk ([Standard Deviation](https://okama.io/#/glossary?page=standard-deviation)) is computed with `risk: TimeSeries` method. It accepts `period` parameter that is either `year` (by default) or `month`:

In [15]:
asset.risk()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17183614]

In [16]:
asset.risk(period='year')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17183614]

In [17]:
asset.risk(period='month')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.04331148]